todo: max size for resulting documents?

In [ ]:
# According to chatgpt:


from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

# Load your documents (tiny txt files)
docs = []
for path in ["doc1.txt", "doc2.txt", "doc3.txt"]:
    with open(path, "r", encoding="utf-8") as f:
        docs.append(f.read().strip())

# Step 1: Embed documents
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(docs, normalize_embeddings=True)

# Step 2: Cluster with HAC (distance_threshold defines merge cutoff)
clustering = AgglomerativeClustering(
    n_clusters=None,                # let threshold decide
    distance_threshold=0.35,        #  Note its 1 - cosine similarity, so lower is stricter!
    affinity="cosine",
    linkage="average"
)
labels = clustering.fit_predict(embeddings)

# Step 3: Merge docs by cluster
from collections import defaultdict
clusters = defaultdict(list)
for label, doc in zip(labels, docs):
    clusters[label].append(doc)

combined_docs = ["\n\n".join(group) for group in clusters.values()]

# Now you have larger documents ready for DeepEval synthesizer
for i, bigdoc in enumerate(combined_docs, 1):
    with open(f"merged_doc_{i}.txt", "w", encoding="utf-8") as f:
        f.write(bigdoc)
